In [1]:
# importing libraries

import pandas as pd
import numpy
import re
from itertools import groupby

In [2]:
# reading excel files into different dataframes

lttc_800=pd.read_excel('../Data/LTTC_data/LTTC_800_Data.xlsx',sheet_name='Paragraph_Data',usecols=['file','Para_No','Sentence_Label','Paragraph_Label','ADU_Flow'])
lttc_800.ADU_Flow=lttc_800.ADU_Flow.replace({'MajorClaim': 'MC'}, regex=True)
file_list=lttc_800[['file']].drop_duplicates().reset_index(drop=True)
temp=file_list.head(3)
lttc_800

,file,Para_No,Paragraph_Label,Sentence_Label,ADU_Flow
0,7165,1,Introduction,"Prompt,Rebuttal,Transition","None,Premise,Premise"
1,7165,2,Body,"Transition,Prompt,Elaboration,Transition","None,Premise,Premise,Premise"
2,7165,3,Conclusion,Suggestion,Premise
3,7168,1,Introduction,"Prompt,Transition,Transition,Transition","None,Premise,Premise,Premise"
4,7168,2,Conclusion,"Prompt,Elaboration,Transition,Prompt,Prompt,El...","Claim,Premise,Premise,Premise,Premise,None,Cla..."
...,...,...,...,...,...
2213,9381,2,Body,"Prompt,Elaboration,Elaboration,Elaboration,Sol...","Claim,Premise,None,Premise,Premise,None,Premis..."
2214,9381,3,Body,"Elaboration,Elaboration,Transition,Transition,...","Premise,Claim,Premise,None,Premise,Premise"
2215,9381,4,Conclusion,Suggestion,Premise
2216,9382,1,Introduction,"Prompt,Elaboration,Conclusion,Transition,Trans...","Claim,Premise,Claim,Claim,Premise,Premise,Prem..."


In [5]:
#---------- extracting statistics for paragraph and essay.

essay_info=pd.DataFrame()
para_info=pd.DataFrame()
for index,row in file_list.iterrows():
    temp_essay=[]
    count_Horizontal=count_Vertical=claims=premise=MC=Non=claim_mc_missing=para_without_adu=count_ADUs=total_units=0
    claim_position=MC_position=MC_present=claims_less_than_50=adu_nonadu_ratio=count_non_adu_para=non_adu_more_than_half_adu=climactic=anti_climactic=0
    Horizontal=Vertical=0
    df=(lttc_800.loc[lttc_800['file']==row['file']]).reset_index(drop=True)
#     print(df)
    total_arguments=len(df)
    for i,r in df.iterrows():
        temp_para=[]
        claims=r.ADU_Flow.count('Claim')
        premise=r.ADU_Flow.count('Premise')
        MC=r.ADU_Flow.count('MC')
        Non=r.ADU_Flow.count('None')
        count_ADUs=claims+premise+MC
        total_units=count_ADUs+Non
        adu_nonadu_ratio=adu_nonadu_ratio+count_ADUs/total_units
        if count_ADUs==0:
            count_non_adu_para=count_non_adu_para+1
        if count_ADUs<(total_units/2):
            non_adu_more_than_half_adu=1
        temp_para.append([row.file,count_ADUs,total_units,non_adu_more_than_half_adu])
        para_info=para_info.append(temp_para)
        if (('Claim,Premise') in r.ADU_Flow or ('MC,Premise') in r.ADU_Flow):
            count_Vertical=count_Vertical+1
        else:
            count_Horizontal=count_Horizontal+1
        if ('Claim' in r.ADU_Flow):
            if claim_position==0:
                claim_position=i+1
                continue
        elif ('MC' in r.ADU_Flow):
            MC_present=True
            if MC_position==0:
                MC_position=i+1
                continue
        else:
            claim_mc_missing=claim_mc_missing+1   
    if claim_mc_missing>(total_arguments/2):
        claims_less_than_50=1
    if MC_position>(total_arguments/2):
        climactic=True
    else:
        anti_climactic=True
    if count_Horizontal>count_Vertical:
        Horizontal=1
    else:
        Vertical=1
    if count_non_adu_para>1:
        para_without_adu=1
    adu_nonadu_ratio=adu_nonadu_ratio/len(df)
    temp_essay.append([row.file,total_arguments,count_Vertical,count_Horizontal,claim_mc_missing,claims_less_than_50
                      ,claim_position,MC_position,climactic,anti_climactic,MC_present,Horizontal,
                       Vertical,para_without_adu,adu_nonadu_ratio])
    essay_info=essay_info.append(temp_essay)

# -----merging culture info with extracted statistics

essay_info.columns=['file','total_arguments','count_Vertical','count_Horizontal',
                    'claim_mc_missing','claims_less_than_50_percent','Claim Position','MC_position',
                    'climactic','anti_climactic','MC_present','Horizontal','Vertical','para_without_adu',
                    'adu_ratio']

essay=essay_info

In [6]:
essay

,file,total_arguments,count_Vertical,count_Horizontal,claim_mc_missing,claims_less_than_50_percent,Claim Position,MC_position,climactic,anti_climactic,MC_present,Horizontal,Vertical,para_without_adu,adu_ratio
0,7165,3,0,3,3,1,0,0,0,1,0,1,0,0,0.805556
0,7168,2,1,1,1,0,2,0,0,1,0,0,1,0,0.750000
0,7169,3,1,2,2,1,1,0,0,1,0,1,0,0,0.875000
0,7171,1,0,1,0,0,1,0,0,1,0,1,0,0,0.900000
0,7176,5,1,4,1,0,1,0,0,1,0,1,0,0,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,9373,2,1,1,1,0,2,0,0,1,0,0,1,0,0.829545
0,9378,4,3,1,1,0,1,4,1,0,1,0,1,0,0.937500
0,9381,4,2,2,2,0,2,0,0,1,0,0,1,0,0.895833
0,9382,1,1,0,0,0,1,0,0,1,0,0,1,0,1.000000


In [7]:
#------------- calculating rations of horizontal and Vertical arguments in each essay
ratio=pd.DataFrame()
for i,r in essay.iterrows():
    temp=[]
    temp.append([r['count_Horizontal']/r['total_arguments'],r['count_Vertical']/r['total_arguments'],r['claim_mc_missing']/r['total_arguments']])
    ratio=ratio.append(temp)
ratio.columns=['H_Ratio','V_Ratio','C_MC_missing_ratio']
essay=pd.concat([essay,ratio],axis=1)
essay

,file,total_arguments,count_Vertical,count_Horizontal,claim_mc_missing,claims_less_than_50_percent,Claim Position,MC_position,climactic,anti_climactic,MC_present,Horizontal,Vertical,para_without_adu,adu_ratio,H_Ratio,V_Ratio,C_MC_missing_ratio
0,7165,3,0,3,3,1,0,0,0,1,0,1,0,0,0.805556,1.000000,0.000000,1.000000
0,7168,2,1,1,1,0,2,0,0,1,0,0,1,0,0.750000,0.500000,0.500000,0.500000
0,7169,3,1,2,2,1,1,0,0,1,0,1,0,0,0.875000,0.666667,0.333333,0.666667
0,7171,1,0,1,0,0,1,0,0,1,0,1,0,0,0.900000,1.000000,0.000000,0.000000
0,7176,5,1,4,1,0,1,0,0,1,0,1,0,0,0.866667,0.800000,0.200000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,9373,2,1,1,1,0,2,0,0,1,0,0,1,0,0.829545,0.500000,0.500000,0.500000
0,9378,4,3,1,1,0,1,4,1,0,1,0,1,0,0.937500,0.250000,0.750000,0.250000
0,9381,4,2,2,2,0,2,0,0,1,0,0,1,0,0.895833,0.500000,0.500000,0.500000
0,9382,1,1,0,0,0,1,0,0,1,0,0,1,0,1.000000,0.000000,1.000000,0.000000


In [13]:
data_lttc_sent=pd.read_excel('../Data/LTTC_data/LTTC_800_Data.xlsx',sheet_name='Sentence_Data',usecols=['file','Text'])
# data_lttc_sent

In [14]:
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from nltk.corpus import wordnet
import nltk
def noOfWords(essay):
    count=0
    for i in essay2word(essay):
        count=count+len(i)
    return count
def sent2word(x):
    x=re.sub("[^A-Za-z0-9]"," ",x)
    words=nltk.word_tokenize(x)
    return words
def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

In [15]:
data_lttc_sent['length']=data_lttc_sent.Text.apply(noOfWords)
avg_sent_len=pd.DataFrame()
for i,r in file_list.iterrows():
    temp=[]
    df=data_lttc_sent.loc[data_lttc_sent['file']==r['file']]
    temp.append([r['file'],df.length.sum()/len(df)])
    avg_sent_len=avg_sent_len.append(temp)
avg_sent_len.columns=['file','Avg_Sent_Len']

In [16]:
avg_sent_len

,file,Avg_Sent_Len
0,7165,18.125000
0,7168,15.833333
0,7169,13.058824
0,7171,17.600000
0,7176,25.636364
...,...,...
0,9373,12.600000
0,9378,10.357143
0,9381,13.850000
0,9382,16.250000


In [17]:
essay=pd.merge(essay,avg_sent_len,on='file',how='left')
essay

,file,total_arguments,count_Vertical,count_Horizontal,claim_mc_missing,claims_less_than_50_percent,Claim Position,MC_position,climactic,anti_climactic,MC_present,Horizontal,Vertical,para_without_adu,adu_ratio,H_Ratio,V_Ratio,C_MC_missing_ratio,Avg_Sent_Len
0,7165,3,0,3,3,1,0,0,0,1,0,1,0,0,0.805556,1.000000,0.000000,1.000000,18.125000
1,7168,2,1,1,1,0,2,0,0,1,0,0,1,0,0.750000,0.500000,0.500000,0.500000,15.833333
2,7169,3,1,2,2,1,1,0,0,1,0,1,0,0,0.875000,0.666667,0.333333,0.666667,13.058824
3,7171,1,0,1,0,0,1,0,0,1,0,1,0,0,0.900000,1.000000,0.000000,0.000000,17.600000
4,7176,5,1,4,1,0,1,0,0,1,0,1,0,0,0.866667,0.800000,0.200000,0.200000,25.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,9373,2,1,1,1,0,2,0,0,1,0,0,1,0,0.829545,0.500000,0.500000,0.500000,12.600000
796,9378,4,3,1,1,0,1,4,1,0,1,0,1,0,0.937500,0.250000,0.750000,0.250000,10.357143
797,9381,4,2,2,2,0,2,0,0,1,0,0,1,0,0.895833,0.500000,0.500000,0.500000,13.850000
798,9382,1,1,0,0,0,1,0,0,1,0,0,1,0,1.000000,0.000000,1.000000,0.000000,16.250000


In [19]:
lttc_arg_stats=pd.DataFrame()
temp=[]
df=essay
temp.append(['Taiwanese',len(df),df.H_Ratio.mean(),df.V_Ratio.mean(),df.C_MC_missing_ratio.mean(),
             df.total_arguments.sum()/len(df),df.climactic.sum()/len(df),df.anti_climactic.sum()/len(df),
             df.Horizontal.sum()/len(df),df.Vertical.sum()/len(df),df.MC_present.sum()/len(df),
             df.claims_less_than_50_percent.sum()/len(df),df.Avg_Sent_Len.sum()/len(df), df.para_without_adu.sum()/len(df),df.adu_ratio.sum()/len(df)])
lttc_arg_stats=lttc_arg_stats.append(temp)

lttc_arg_stats.columns=['llanguage','#Essays','H_Ratio','V_Ratio','C_MC_missing_ratio',
                           'avg_no_arguments_per_essay','ratio_Climactic_essays','ratio_Anti_Climactic_essays',
                           'Ratio_Horizontal_arguments','Ratio_Vertical_arguments','ratio_essay_with MC',
                           'Claims_less_than_50%','Avg_Sentence_Length','para_without_adu','adu_ratio']
lttc_arg_stats

,llanguage,#Essays,H_Ratio,V_Ratio,C_MC_missing_ratio,avg_no_arguments_per_essay,ratio_Climactic_essays,ratio_Anti_Climactic_essays,Ratio_Horizontal_arguments,Ratio_Vertical_arguments,ratio_essay_with MC,Claims_less_than_50%,Avg_Sentence_Length,para_without_adu,adu_ratio
0,Taiwanese,800,0.551818,0.448182,0.329979,2.7725,0.0475,0.9525,0.49875,0.50125,0.055,0.22875,17.124485,0.00125,0.835385


In [20]:
lttc_arg_stats.to_excel('lttc_arg_stats_v2.xlsx')

In [ ]:
para_info.columns=['file','#ADU_Units','Total_Units','non_adu_more_than_half_adu']
# para=pd.merge(culture_info,para_info,on='file',how='left')
para_info.non_adu_more_than_half_adu.value_counts()